In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from utility import calculate_input_gradients, perturb_inputs, preprocess_images, \
                    postprocess_features, save_data_hdf5,get_dataset_hdf5,\
                    build_one_class_svm, combine_inliners_outliers, apply_temp_scale_to_model,\
                    apply_log_temp_scale_to_model, perturb_inputs_odin, extract_layer_features
 
from utility_db_outliers import load_dataset
from models_lib import load_custom_model_for_ds
import h5py
# from metrics2 import *
from metrics import *
from general_setting import *
import time
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
#------------
from utility_methods import *
# from utility_methods2 import *
from sklearn.metrics import roc_auc_score

from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import mnist
import scipy.io as sio
import tensorflow.keras.backend as K
import math

In [2]:
import matplotlib.pyplot as plt
from scipy.stats import norm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv2D, Flatten, Lambda
from tensorflow.keras.layers import Reshape, Conv2DTranspose, MaxPooling2D, UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [3]:
BATCH_SIZE = 100
SAVE_RESULTS = True
id_name=ID_DS_LIST[0] # selects the ID dataset.
id_model=ID_MODEL_LIST[0]  # select the deep model used for training ID dataset.
print(id_name,id_model)

MNIST LeNet


In [4]:
org_model = load_custom_model_for_ds(id_name, id_model)

The weights of CUSTOM-MNIST model was loaded.


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import (Conv2D, Conv2DTranspose, Dense, Flatten,
                                     InputLayer, Reshape, Layer, Input)
from typing import Callable, Dict, List, Tuple, Union
from tensorflow.keras.losses import kld, categorical_crossentropy


In [6]:
base_model = org_model

In [7]:
def head_model():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation='relu',strides=2, padding='same', input_shape=(28, 28, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu',strides=2, padding='same'))
    model.add(Conv2DTranspose(16, (3, 3), activation='relu',strides=2, padding='same'))
    model.add(Conv2DTranspose(1, (3, 3), activation='sigmoid',strides=2, padding='same'))  
    return model


In [8]:
head_model = head_model()
head_model.add(base_model)
model = head_model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 16)        4624      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 1)         145       
_________________________________________________________________
sequential_3 (Sequential)    (None, 10)                44426     
Total params: 53,995
Trainable params: 53,995
Non-trainable params: 0
_________________________________________________________________


In [9]:
(org_traing_data, org_training_labels),(id_eva_data,org_testing_labels)  = load_dataset(id_name)

org_traing_data_processed = preprocess_images(id_name, org_traing_data, id_model, verbose=True)
id_eva_data_processed = preprocess_images(id_name, id_eva_data, id_model, verbose=True)

Preprocessing is not needed.
Preprocessing is not needed.


/home/zhaofz/anaconda3/envs/tf-gpu3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [10]:
x_train = org_traing_data_processed
y_train = org_training_labels
x_test = id_eva_data_processed
y_test = org_testing_labels

In [ ]:
y_true = base_model.predict(x_train)
print(y_true.shape)
np.save('./KL_datas/%s_%s_train.npy'%(id_name, id_model), x_train)
np.save('./KL_datas/%s_%s_preds.npy'%(id_name, id_model), y_true)

In [11]:
x_train= np.load('./KL_datas/%s_%s_train.npy'%(id_name, id_model))
y_true = np.load('./KL_datas/%s_%s_preds.npy'%(id_name, id_model))

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler( )
scaler.fit(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test_0 = scaler.transform(x_test.flatten().reshape(x_test.shape[0],x_test.shape[1]*x_test.shape[2]*x_test.shape[3]))
x_test = x_test_0.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],x_test.shape[3])

In [ ]:
scaler.fit(x_train.flatten().reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]*x_train.shape[3]))
x_train_0 = scaler.transform(x_train.flatten().reshape(x_train.shape[0],x_train.shape[1]*x_train.shape[2]*x_train.shape[3]))
x_train = x_train_0.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],x_train.shape[3])

In [ ]:
y_test = to_categorical(y_test,10)
sc = base_model.evaluate(x_test,y_test)
print(sc)

In [ ]:
for layer in model.layers:
    if str(layer.name) == 'sequential':
        layer.trainable = False

In [14]:
def my_loss_fn(y_true, y_pred):
    temperature = 150
    y_true = y_true ** (1 / temperature)
    y_true = y_true / tf.reshape(tf.reduce_sum(y_true, axis=-1), (-1, 1))
    loss_kld = kld(y_true, y_pred)
    std_kld = tf.math.reduce_std(loss_kld)
    loss = tf.reduce_mean(loss_kld)
    return loss

In [ ]:
y_train = y_true

In [ ]:
model.compile(loss=my_loss_fn, optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=256, epochs=30)
model.save('./KL_models/%s_%s_KL.h5'%(id_name, id_model))

In [15]:
model = load_model('./KL_models/%s_%s_KL.h5'%(id_name,id_model), custom_objects={'my_loss_fn': my_loss_fn})
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 14, 14, 16)        4624      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 28, 28, 1)         145       
_________________________________________________________________
sequential_3 (Sequential)    (None, 10)                44426     
Total params: 53,995
Trainable params: 53,995
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x0 = base_model.predict(x_test[0].reshape(1,28,28,1))
x1 = model.predict(x_test[0].reshape(1,28,28,1))

In [ ]:
feature_model = Model(inputs=model.input, outputs=model.layers[-2].output)

In [ ]:
img1 = feature_model.predict(x_test[0].reshape(1,28,28,1))
print(img1.shape)
plt.imshow(img1.squeeze())
plt.show()

In [ ]:
a = my_loss_fn(x0,x1)
print(a)

In [ ]:
das = np.load('./advs_new/%s_%s_cw-l2.npy'%(id_name, id_model),allow_pickle=True)

In [ ]:
y0 = base_model.predict(das[0].reshape(1,28,28,1))
y1 = model.predict(das[0].reshape(1,28,28,1))
b = my_loss_fn(y0,y1)
print(b)

In [16]:
def mark(y_true, y_pred):
    temperature = 150
    y_true = y_true ** (1 / temperature)
    loss_kld = kld(y_true, y_pred)
    std_kld = tf.math.reduce_std(loss_kld)
    loss = tf.reduce_mean(loss_kld)
    return loss

In [22]:
x_test = np.load('./advs_new/%s_%s_normal.npy'%(id_name, id_model),allow_pickle=True)

print(x_test.shape)

(9920, 28, 28, 1)


In [23]:
sc_id = []
for i in range(int(len(x_test))):
    x0 = base_model.predict(x_test[i].reshape(1,28,28,1))
    x1 = model.predict(x_test[i].reshape(1,28,28,1))
    score = mark(x0,x1)
    # print(score)
    sc_id.append(-score)

In [18]:
def combine_inliners_outliers(inliers, outliers, i_label=1, o_label=0, verbose=False):

    temp_outliers = outliers
    temp_inliers  = inliers
    if i_label==1:
        i_labels = np.ones(temp_inliers.shape[0])
    else:
        i_labels = np.zeros(temp_inliers.shape[0])
        
    if o_label==0:
        o_labels = np.zeros(temp_outliers.shape[0])
    else:
        o_labels = np.ones(temp_outliers.shape[0])       
              
    mixed_labels =  np.append(i_labels, o_labels)
    mixed_data = np.hstack((temp_inliers, temp_outliers))

    return mixed_data, mixed_labels

In [24]:
test = np.asarray(sc_id)
ADV_DS_LIST = ("fgsm", "bim-a", "bim-b", "jsma", "cw-l2")
for i, ds_name in enumerate(ADV_DS_LIST):
    adv = id_name+"_"+id_model+"_"+ ds_name
    das = np.load('./advs_new/'+adv+'.npy',allow_pickle=True)
    
    # scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        x0 = base_model.predict(das[i].reshape(1,28,28,1))
        x1 = model.predict(das[i].reshape(1,28,28,1))
        score = mark(x0,x1)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood)
    scores, mixed_labels = combine_inliners_outliers(test, ood)
 
    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]

    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
    #     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ds_name))
    #     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
    #     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

The results of fgsm:

tnr_at_95_tpr: 91.78861788524605
detection_acc: 93.39430894214802
AUROC:  98.21483511235041


The results of bim-a:

tnr_at_95_tpr: 82.5203252024134
detection_acc: 88.76016260073169
AUROC:  97.20757084905047


The results of bim-b:

tnr_at_95_tpr: 1.8902439024198143
detection_acc: 48.4451219507349
AUROC:  37.335716985887096


The results of jsma:

tnr_at_95_tpr: 91.71747967386466
detection_acc: 93.35873983645733
AUROC:  98.44079704165583


The results of cw-l2:

tnr_at_95_tpr: 68.3434959342648
detection_acc: 81.67174796665739
AUROC:  95.08471755902055




In [19]:
#layer_inx specifies the index of the OODL found by "find_oodl" jupyter file.
REM_TOP_LAYER = -2
NUM_CLASS = 10
if id_name=="MNIST" and id_model=="LeNet":
    layer_inx = 0
    OOD_DS_LIST       = OOD_DS_LIST_MNIST
    #**********************************************************
elif id_name=="CIFAR10":
    OOD_DS_LIST      = OOD_DS_LIST_CIFAR10
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 20
    #**********************************************************
elif id_name=="SVHN":
    OOD_DS_LIST      = OOD_DS_LIST_SVHN
    if id_model=="VGG":
        layer_inx = 6
    elif id_model=="ResNet":
        layer_inx = 24

In [20]:
# Vriance模块(OODL大类别 --> 删除同样的feature maps)
test = np.asarray(sc_id)
for i, ood_ds_name in enumerate(OOD_DS_LIST):
    (_,_),(ood_eva_data,_) = load_dataset(ood_ds_name)
    das = preprocess_images(id_name, ood_eva_data, id_model, verbose=True)
    
    # scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
    # das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])
    
    sc_ood = []
    for i in range(len(das)):
        x0 = base_model.predict(das[i].reshape(1,28,28,1))
        x1 = model.predict(das[i].reshape(1,28,28,1))
        score = mark(x0,x1)
        sc_ood.append(-score)
    
    ood = np.asarray(sc_ood) 
    scores, mixed_labels = combine_inliners_outliers(test, ood) 

    fpr, tpr = nums(scores, mixed_labels)
    roc_auc = auc(fpr, tpr)
    lens = id_eva_data_processed.shape[0]
    FP,TN,TP,FN = ErrorRateAt95Recall1(lens, scores, mixed_labels)
    ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
#     print(get_summary_statistics(scores, mixed_labels))
    print('The results of %s:\n' %(ood_ds_name))
#     print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
    print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
    print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
#     print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
    print("AUROC: ",ROC*100)
    print("\n")

Preprocessing is not needed.
The results of FASHION_MNIST:

tnr_at_95_tpr: 99.599999999004
detection_acc: 97.299999999027
AUROC:  99.81613549999999


Preprocessing is not needed.
The results of OMNIGLOT_RESIZED_28:

tnr_at_95_tpr: 99.85999999900139
detection_acc: 97.42999999902568
AUROC:  99.8312435


Preprocessing is not needed.
The results of MNIST_GAUSSIAN_NOISE_ODIN:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  100.0


Preprocessing is not needed.
The results of MNIST_UNIFORM_NOISE:

tnr_at_95_tpr: 99.99999999899998
detection_acc: 97.499999999025
AUROC:  99.9522795




In [21]:
ood_ds_name = 'foolingimages'

das = np.load('adv_datasets/%s_%s_fooling_images.npy'%(id_name,id_model))

# scaler.fit(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das_0 = scaler.transform(das.flatten().reshape(das.shape[0],das.shape[1]*das.shape[2]*das.shape[3]))
# das = das_0.reshape(das.shape[0],das.shape[1],das.shape[2],das.shape[3])

sc_ood = []
for i in range(len(das)):
        x0 = base_model.predict(das[i].reshape(1,28,28,1))
        x1 = model.predict(das[i].reshape(1,28,28,1))
        score = mark(x0,x1)
        sc_ood.append(-score)

ood = np.asarray(sc_ood) 
scores, mixed_labels = combine_inliners_outliers(test, ood) 

# print('scores is:', (scores[10000:20000]))
fpr, tpr = nums(scores, mixed_labels)
roc_auc = auc(fpr, tpr)
lens = id_eva_data_processed.shape[0]

FP,TN,TP,FN = ErrorRateAt95Recall1(lens,scores, mixed_labels)
ROC = roc_auc_score(mixed_labels, scores, average='micro', sample_weight=None)
print('The results of %s:\n' %(ood_ds_name))
# print(get_summary_statistics(scores, mixed_labels))
# print('fpr_at_95_tpr:', float(FP) / float(FP + TN+ 1e-7) *100)
print('tnr_at_95_tpr:', float(TN) / float(FP + TN+ 1e-7) *100)
print('detection_acc:',(float(TP) / float(TP + FN + 1e-7)+ float(TN) / float(FP + TN + 1e-7))/2*100)
# print('detection_errror:',(1.0- float(TP) / float(TP + FN+ 1e-7)+ float(FP) / float(FP + TN+ 1e-7))/2*100)
print("AUROC: ",ROC*100)
print("\n")

The results of foolingimages:

tnr_at_95_tpr: 0.0
detection_acc: 47.499999999524995
AUROC:  86.841421


